In [168]:
import os
from glob import glob # extract path of each file
import pandas as pd # data preprocessing
from xml.etree import ElementTree as et # parse information from XML
from functools import reduce

In [169]:
import warnings
warnings.filterwarnings('ignore')

In [170]:
# step-1: get path of each xml file
xmlfiles = glob('./data_images/*.xml')
# replace \\ with /
replace_text = lambda x: x.replace('\\','/')
xmlfiles = list(map(replace_text,xmlfiles))

In [171]:
xmlfiles

['./data_images/Sasakyan02986.xml']

In [172]:
# step-2: read xml files
# from each xml file we need to extract
# filename, size(width, height), object(name, xmin, xmax, ymin, ymax)
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()

    # extract filename
    image_name = root.find('filename').text
    # width and height of the image
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name, width, height, name,xmin,xmax,ymin,ymax])
        
    return parser

In [173]:
parser_all = list(map(extract_text,xmlfiles))

In [174]:
data = reduce(lambda x, y : x+y,parser_all)

In [175]:
df = pd.DataFrame(data,columns = ['filename','width','height','name','xmin','xmax','ymin','ymax'])

In [176]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax


In [177]:
df.shape

(0, 8)

In [178]:
df['name'].value_counts()

Series([], Name: count, dtype: int64)

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  0 non-null      object
 1   width     0 non-null      object
 2   height    0 non-null      object
 3   name      0 non-null      object
 4   xmin      0 non-null      object
 5   xmax      0 non-null      object
 6   ymin      0 non-null      object
 7   ymax      0 non-null      object
dtypes: object(8)
memory usage: 124.0+ bytes


In [180]:
# type conversion
cols = ['width','height','xmin','xmax','ymin','ymax']
df[cols] = df[cols].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  0 non-null      object
 1   width     0 non-null      int64 
 2   height    0 non-null      int64 
 3   name      0 non-null      object
 4   xmin      0 non-null      int64 
 5   xmax      0 non-null      int64 
 6   ymin      0 non-null      int64 
 7   ymax      0 non-null      int64 
dtypes: int64(6), object(2)
memory usage: 124.0+ bytes


In [181]:
# center x, center y
df['center_x'] = ((df['xmax']+df['xmin'])/2)/df['width']
df['center_y'] = ((df['ymax']+df['ymin'])/2)/df['height']
# w 
df['w'] = (df['xmax']-df['xmin'])/df['width']
# h 
df['h'] = (df['ymax']-df['ymin'])/df['height']

In [182]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h


In [183]:
images = df['filename'].unique()

In [184]:
len(images)

0

In [185]:
# 80% train and 20% test
img_df = pd.DataFrame(images,columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename']) # shuffle and pick 80% of images

In [186]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename']) # take rest 20% images

In [187]:
len(img_train), len(img_test)

(0, 0)

In [188]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')

In [189]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h


In [190]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h


In [191]:
def label_encoding(x):
    labels = {'car':0, 'motorcycle':1, 'truck':2, 'bus':3, 'jeep':4, 'tricycle':5, 'face':6 }
    return labels[x]

In [192]:
train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

In [193]:
train_df.head(10)

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id


In [194]:
import os
from shutil import move

In [195]:
train_folder = 'data_images/train'
test_folder = 'data_images/test'


In [196]:
cols = ['filename','id','center_x','center_y', 'w', 'h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

In [197]:
#groupby_obj_train.get_group('000009.jpg').set_index('filename').to_csv('sample.txt',index=False,header=False)
# save each image in train/test folder and repective labels in .txt
def save_data(filename, folder_path, group_obj):
    # move image
    src = os.path.join('data_images',filename)
    dst = os.path.join(folder_path,filename)
    move(src,dst) # move image to the destination folder
    
    # save the labels
    text_filename = os.path.join(folder_path,
                                 os.path.splitext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index=False,header=False)
    

In [198]:
filename_series = pd.Series(groupby_obj_train.groups.keys())

In [199]:
filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

Series([], dtype: object)